# Azuure Sentinel Notebooks -Introduction to Sentinel Code Snippets

Azure Sentinel Notebook features enables Security Analysts to leverage Azure Notebooks to excute code and kql driven analysis. Notebooks are built to reflect most of the Open Source Jupyter Notebook capabilities.  
The Jupyter Notebook (formerly IPython Notebooks) is defined by Wikipedia as an Open Source web-based interactive computational environment for creating notebook documents. It can be classified as a programming tool.  
A Jupyter Notebook document is a browser-based REPL containing an ordered list of input/output cells which can contain code, text (using Markdown), mathematics, plots and rich media.  
Underneath the interface, a notebook is a JSON document, following a versioned schema, usually ending with the ".ipynb" extension.  
Before using a Notebook, you're required to provision a compute system built with the Kernles and default runtimes to work with the Notebook.  
[Azure Sentinel Notebooks Ninja Part 2: Getting Started with Azure Sentinel Notebooks](https://techcommunity.microsoft.com/t5/azure-sentinel/azure-sentinel-notebooks-ninja-part-2-getting-started-with-azure/ba-p/2716661) will provide how to start with Azure Sentinel Notebook.  

This Notebook is developed to provide you with some of the basic and essential code to create your first custom Notebook. It uses built-in Code Snippets that you can access by simply typing the word "Sentinel".

## Sample notebook that uses Azure Sentinel code snippets
Code Snippets are convenient and saves times when performing the same task. If you use snippets, you can significantly reduce time, organize many processes, and get a ready-made solution for many tasks.  
This concept applies to Sentinel Notebooks as well. When comes down to develop a notebook for Sentinel, you will need four fundamental pieces of code that allow the notebook to authenticate and connect to the data source stored on the workspace, load all the packages required to formulate and execute KQL queries against it, and setup the environment for the integration with Microsoft Security Center (mstic) Python packages (msticpy).  

**By typing the  keyword “Sentinel” you’ll be able to import the needed code for the above requirements** 
 
There four Code Snippets are:  
 
1. Get Configuration parameters  
2. Set up environment for msticpy  
3. Authenticate into Azure resources  
4. Authenticate into Azure Log Analytics   


# First custom Sentinel Notebook (Jupyter)
---
There are at least 3 ways you can connect, query, and consume data from Sentinel.  
1. Using Microsoft msticpy    
2. Using LogAnalyticsManagementClient  
3. Using KQL Magic

---


If they don't have it, then they will need to create this file from scratch, like the steps you show.



### Create a config.json file
---
The config.json file is used to set the configurations parameters that you can use when connecting to Azure Sentinel.  
This file is automatically generated and properly configured when you create the Notebook Compute system from Sentinel and imported at least one of the Notebooks available from the template library.  
The file will have all the value properly added based on the Sentinel workspace you've used for the Notebook, the Azure Subscription and Resource Group where Sentinel is located, the Tenant ID, and both the user alias and object ID.
In the case where you're using a custom Compute system for your Notebooks then you can create this file manually. 

-**Here how to create the file manually**- 

Create a new file under your directory and paste the following.  
Edit the information as relates to your Azure Subscription, Tenant ID, Log Analytic Workspace used by Sentinel, your accout Alias and your account ID

`
{
    "tenant_id":"<Azure_Active_Directory_Tenant_Id>",  
    "subscription_id":"<Azure_Subscription_Id?",  
    "resource_group":"<Sentinel_Resource_Group>",  
    "workspace_id":"<Sentinel_Workspace_id>",  
    "workspace_name":"<Sentinel_Worspace_Name>",  
    "user_alias":"<User_Alias_With_Permissions_to_Access_Sentinel>",  
    "user_object_id":"<User_Object_Id>"  
}
`


## 1. Using Microsoft MSTICPy
---
 


### Installing MSTICPy using a Code Snippet

The Microsoft Threat Intelligence Center (MSTIC) team is one of the main producers of threat intelligence at Microsoft as it collects the threat intelligence that’s infused into products and services.  
MSTICPy is a library for InfoSec investigation and hunting in Jupyter Notebooks. It includes functionality to:  
* query log data from multiple sources  
* enrich the data with Threat Intelligence, geolocations and Azure resource data  
* extract Indicators of Activity (IoA) from logs and unpack encoded data  
* perform sophisticated analysis such as anomalous session detection and time series decomposition  
* visualize data using interactive timelines, process trees and multi-dimensional Morph Charts  

It also includes some time-saving notebook tools such as widgets to set query time boundaries, select and display items from lists, and configure the notebook environment, as well as Python library to query Sentinel via KQL (i.e. Kqlmagic).


To use MSTICPy, you'll need to install it first since the Compute Kernel does not include it by default.  
The below code cell uses one of the four Code Snippet described early but it includes a tiny variant and that is this: `%pip install --upgrade --quiet msticpy[azuresentinel,ml]`  
By including the **"azuresentinel,ml"** in the command you'll install the modules for Azure, Sentinel, Key Vault, KQL, and ML:

- Key Vault and keyring storage of settings secrets  
- Azure Sentinel data queries  
- Kqlmagic Jupyter extended  
- Azure API data retrieval (subs, resources, Vms, etc.)  
- Azure storage APIs  
- Azure Sentinel APIs (not data query)   
- Timeseries analysis  
- Event clustering  
- Outlier analysis 

In [ ]:
#####################################################
# Sentinel - Set up environment for msticpy         #
#####################################################
# import some modules needed in this cell
from pathlib import Path
from IPython.display import display, HTML

display(HTML('Checking upgrade to latest msticpy version'))
%pip install --upgrade --quiet msticpy[azuresentinel,ml]

REQ_PYTHON_VER=(3, 6)
REQ_MSTICPY_VER=(1, 2, 3)
REQ_MP_EXTRAS = ['keyvault']

display(HTML('<h3>Starting Notebook setup...</h3>'))
if Path('./utils/nb_check.py'):
    from utils.nb_check import check_versions
    check_versions(REQ_PYTHON_VER, REQ_MSTICPY_VER)

# intialize msticpy
from msticpy.nbtools import nbinit
nbinit.init_notebook(
    namespace=globals(),
    extra_imports=['urllib.request, urlretrieve']
)
pd.set_option('display.html.table_schema', False)

### Configure msticpyconfig.yaml
MSTICPy uses a yaml configuration file **msticpyconfig.yaml**
To generate the config file you'll need the config.json file first, then execute the below code cell to generate the yaml file and configure it if you wish to add other seting beside the Default.


In [3]:
#####################################################
# MSTICPy - Set up msticpyconfig.yaml #
#####################################################
from msticpy.config import MpConfigEdit
import os
mp_conf = "msticpyconfig.yaml"
# check if MSTICPYCONFIG is already an env variable
mp_env = os.environ.get("MSTICPYCONFIG")
mp_conf = mp_env if mp_env and Path(mp_env).is_file() else mp_conf
if not Path(mp_conf).is_file():
    print(
        "Please check that there is a config.json file in your workspace folder.",
        "If this is not there, create it first (sample in Create a config.json file",
        sep="\n"
    )
else:
    mpedit = MpConfigEdit(mp_conf)
    mpedit.set_tab("AzureSentinel")
    display(mpedit)

### Import MSTICPy settings from the msticpyconfig.yaml

In [4]:
#####################################################
# MSTICPy - Import msticpyconfig.yaml #
#####################################################
import msticpy
msticpy.settings.refresh_config()

### Connecting to Sentinel

In [5]:
#####################################################
# MSTICPy - Connect to Sentinel #
#####################################################
## Create a QueryProvider object for running queries in our LogAnalytics workspace
qry_prov = QueryProvider(data_environment='AzureSentinel')
## Use the workspace configuration we've set up in msticpyconfig.yaml (you can also choose another workspace here)
ws_config = WorkspaceConfig(workspace="Default")
qry_prov.connect(connection_str=ws_config.code_connect_str)

Runtime dependency of PyGObject is missing.
Depends on your Linux distro, you could install it system-wide by something like:
    sudo apt install python3-gi python3-gi-cairo gir1.2-secret-1
If necessary, please refer to PyGObject's doc:
https://pygobject.readthedocs.io/en/latest/getting_started.html
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/msal_extensions/libsecret.py", line 21, in <module>
    import gi  # https://github.com/AzureAD/microsoft-authentication-extensions-for-python/wiki/Encryption-on-Linux
ModuleNotFoundError: No module named 'gi'


Please wait. Loading Kqlmagic extension...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

done


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

connected


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Write your KQL query, execute it, and present results

In [ ]:
#####################################################
# MSTICPy - Read data from Sentinel using KQL #
#####################################################
Query_Data_Log_Type = '''
Event | take 10 | where TimeGenerated >=ago(30d)
'''
Results_df = qry_prov.exec_query(Query_Data_Log_Type)
Results_df.head(10)

## 2. Using LogAnalyticsManagementClient
---
This is the Microsoft Azure Log Analytics Management Client Library.

### Getting Sentinel configuration parameters from the config.json file
Before connecting to Sentinel and start querying data we need to initialize Sentinel parameters referred by the rest of the code.  
These parameters are:  
- Azure Tenant
- Azure Subscription ID
- Resource Group
- Workspace ID
- Workspace Name
- User Alias
- USer object ID  

For this task we're going to use a second Sentinel Code Snippet: **Get Configuration Parameters**

In [7]:
####################################################
# Sentinel - Get Sentinel Configuration parameters #
####################################################
import json
def read_config_values(file_path):
    'This loads pre-generated parameters for Azure Sentinel Workspace'
    with open(file_path) as json_file:
        if json_file:
            json_config = json.load(json_file)
            return (json_config['tenant_id'],
                json_config['subscription_id'],
                json_config['resource_group'],
                json_config['workspace_id'],
                json_config['workspace_name'],
                json_config['user_alias'],
                json_config['user_object_id'])
    return None

# Calling the above function to populate Sentinel workspace parameters
tenant_id, subscription_id, resource_group, workspace_id, workspace_name, user_alias, user_object_id = read_config_values('config.json')

### Login and authenticate into Azure Sentinel
To login and authenticate into Azure Sentinel we can use the Sentinel Code Snippet **Authenticate into Azure Log Analytics**.  

You might like to have better background color for the AZ LOGIN command. Use: !echo -e '\e[41m'

The followings are possible color combination:

- BgBlack = '\e[40m' 
- BgRed = '\e[41m' 
- BgGreen = '\e[42m' 
- BgYellow = '\e[43m' 
- BgBlue = '\e[44m' 
- BgMagenta = '\e[45m' 
- BgCyan = '\e[46m' 
- BgWhite = '\e[47m'

In [12]:
####################################################
# Sentinel - Authenticate into Azure Log Analytics #
####################################################
# Azure CLI is used to get device code to login into Azure, you need to copy the code and open the DeviceLogin site.
# !!! You need [tenant_id] and [subscription_id] to login into Azure !!!
from azure.common.client_factory import get_client_from_cli_profile
from azure.common.credentials import get_azure_cli_credentials
from azure.loganalytics import LogAnalyticsDataClient
from azure.mgmt.loganalytics import LogAnalyticsManagementClient
from azure.loganalytics.models import QueryBody

!echo -e '\e[41m'
!az login --use-device-code
la_client = get_client_from_cli_profile(LogAnalyticsManagementClient, subscription_id = subscription_id)
creds, _ = get_azure_cli_credentials(resource='https://api.loganalytics.io')
la_data_client = LogAnalyticsDataClient(creds)


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F8GR29VRB to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "4776646d-711e-4b14-953b-fab89067d5e3",
    "id": "28b4a812-1278-403e-b2ae-e56d086bc019",
    "isDefault": false,
    "managedByTenants": [
      {
        "tenantId": "c25c3a0d-c0be-4f12-b185-16a6fe89e174"
      }
    ],
    "name": "AIA External - WIZLABS",
    "state": "Enabled",
    "tenantId": "4776646d-711e-4b14-953b-fab89067d5e3",
    "user": {
      "name": "gastori@wizlabs.net",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "4776646d-711e-4b14-953b-fab89067d5e3",
    "id": "723a5ed2-45cd-4d12-8880-2f94f7184995",
    "isDefault": false,
    "managedByTenants": [
      {
        "tenantId": "c25c3a0d-c0be-4f12-b185-16a6fe89e174"
      }
    ],
    "name": "AIA External - CYBERLABS",
    "state": "Enabled",
    "tenantId": "4776646d-711e-4b14-953b-fab89067d5

### Write your own query and execute it
Now you can run a quick query using the log analytics data client

In [15]:
query = 'Event | take 10 | where TimeGenerated >=ago(24h)'
result = la_data_client.query(workspace_id, QueryBody(query=query))
print(result.as_dict())


{'tables': [{'name': 'PrimaryResult', 'columns': [{'name': 'TenantId', 'type': 'string'}, {'name': 'SourceSystem', 'type': 'string'}, {'name': 'TimeGenerated', 'type': 'datetime'}, {'name': 'Source', 'type': 'string'}, {'name': 'EventLog', 'type': 'string'}, {'name': 'Computer', 'type': 'string'}, {'name': 'EventLevel', 'type': 'int'}, {'name': 'EventLevelName', 'type': 'string'}, {'name': 'ParameterXml', 'type': 'string'}, {'name': 'EventData', 'type': 'string'}, {'name': 'EventID', 'type': 'int'}, {'name': 'RenderedDescription', 'type': 'string'}, {'name': 'AzureDeploymentID', 'type': 'string'}, {'name': 'Role', 'type': 'string'}, {'name': 'EventCategory', 'type': 'int'}, {'name': 'UserName', 'type': 'string'}, {'name': 'Message', 'type': 'string'}, {'name': 'MG', 'type': 'string'}, {'name': 'ManagementGroupName', 'type': 'string'}, {'name': 'Type', 'type': 'string'}, {'name': '_ResourceId', 'type': 'string'}], 'rows': []}]}


## Using KQL Magic
---
Kqlmagic magic extension enables notebook experience, exploring Microsoft Azure Monitor data: 
- Azure Data Explorer (Kusto)
- ApplicationInsights
- LogAnalytics 

data from Jupyter notebook (Python3 kernel), using kql (Kusto Query language).  
https://pypi.org/project/Kqlmagic/

### Install or Upgrade and Load KQLMAGIC
Execute the next Code Cell to install KQLMagic module. This is needed only if you have not installed MSTICPy.  
The KQLMAGIC_AZUREML_COMPUTE environment variable sets the name of the Azure ML Compute. You can refer to the compute name by looking either at the Compute section of ML Studio or simply in the above Notebook ribbon where it says Compute.  
Take note of the name and enter its value in the below code variable **azml_compute**


In [89]:
# User Input
azml_compute='vm-sentinel-ml01'
kqlmagic_azml_compute='https://' + azml_compute + '.eastus.instances.azureml.ms'

In [90]:
#!pip install Kqlmagic --no-cache-dir --upgrade
%env KQLMAGIC_AZUREML_COMPUTE="$kqlmagic_azml_compute"
%env KQLMAGIC_CONFIGURATION="show_init_banner=False"
%reload_ext Kqlmagic

env: KQLMAGIC_AZUREML_COMPUTE="https://vm-sentinel-ml01.eastus.instances.azureml.ms"
env: KQLMAGIC_CONFIGURATION="show_init_banner=False"


### Getting Sentinel configuration parameters from the config.json file

Before connecting to Sentinel and start querying data is recommended to initialize Sentinel parameter:

    Azure Tenant
    Azure Subscription ID
    Resource Group
    Workspace ID
    Workspace Name
    User Alias
    USer object ID

Use the Sentinel Code Snippet **Get Configuration Parameters**

In [77]:
####################################################
# Sentinel - Get Sentinel Configuration parameters #
####################################################
import json
def read_config_values(file_path):
    'This loads pre-generated parameters for Azure Sentinel Workspace'
    with open(file_path) as json_file:
        if json_file:
            json_config = json.load(json_file)
            return json_config['workspace_id']
    return None

# Calling the above function to populate Sentinel workspace parameters
workspace_id = read_config_values('config.json')

### Login and authenticate into the Worksapce

In [78]:
%kql loganalytics://code;workspace=workspace_id

<IPython.core.display.Javascript object>

### Write your own query and execute it 

In [79]:
%%kql
Event
| take 1
| where TimeGenerated >=ago(30d)

<IPython.core.display.Javascript object>

TenantId,SourceSystem,TimeGenerated,Source,EventLog,Computer,EventLevel,EventLevelName,ParameterXml,EventData,EventID,RenderedDescription,AzureDeploymentID,Role,EventCategory,UserName,Message,MG,ManagementGroupName,Type,_ResourceId
2d868894-04f8-4983-a155-36025f3ee696,OpsManager,2021-09-13 20:26:31.557000+00:00,Microsoft-Windows-Sysmon,Microsoft-Windows-Sysmon/Operational,win7.cyberhuntlabs.com,4,Information,"<Param>technique_id=T1036,technique_name=Masquerading</Param><Param>2021-09-13 20:26:31.553</Param><Param>{16889A29-B3F7-613F-3908-000000003600}</Param><Param>3308</Param><Param>C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe</Param><Param>1.4.00.22472</Param><Param>Microsoft Teams</Param><Param>Microsoft Teams</Param><Param>Microsoft Corporation</Param><Param>Teams.exe</Param><Param>""C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe"" --type=renderer --autoplay-policy=no-user-gesture-required --disable-background-timer-throttling --field-trial-handle=1032,11197078799516264773,3373066149836992762,131072 --enable-features=ContextBridgeMutability,WebComponentsV0Enabled --disable-features=CookiesWithoutSameSiteMustBeSecure,SameSiteByDefaultCookies,SpareRendererForSitePerProcess --disable-gpu-compositing --lang=en-US --enable-wer --ms-teams-less-cors=522133263 --app-user-model-id=com.squirrel.Teams.Teams --app-path=""C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\resources\app.asar"" --device-scale-factor=1 --num-raster-threads=1 --renderer-client-id=338 --no-v8-untrusted-code-mitigations --mojo-platform-channel-handle=3892 /prefetch:1</Param><Param>C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\</Param><Param>CYBERHUNTLABS\nmartha</Param><Param>{16889A29-2C20-613F-201E-130000000000}</Param><Param>0x131e20</Param><Param>2</Param><Param>Low</Param><Param>SHA256=AA09158C973672F5DC092403CBEA41A8E88DC598DCE1D4DC26A7B62C94C97CE5</Param><Param>{16889A29-2C27-613F-1A01-000000003600}</Param><Param>4484</Param><Param>C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe</Param><Param>""C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe"" --system-initiated</Param>","<DataItem type=""System.XmlData"" time=""2021-09-13T20:26:31.5571847+00:00"" sourceHealthServiceId=""906B6E0A-5215-7CA5-3DF1-EFEE04E01033""><EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event""><Data Name=""RuleName"">technique_id=T1036,technique_name=Masquerading</Data><Data Name=""UtcTime"">2021-09-13 20:26:31.553</Data><Data Name=""ProcessGuid"">{16889A29-B3F7-613F-3908-000000003600}</Data><Data Name=""ProcessId"">3308</Data><Data Name=""Image"">C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe</Data><Data Name=""FileVersion"">1.4.00.22472</Data><Data Name=""Description"">Microsoft Teams</Data><Data Name=""Product"">Microsoft Teams</Data><Data Name=""Company"">Microsoft Corporation</Data><Data Name=""OriginalFileName"">Teams.exe</Data><Data Name=""CommandLine"">""C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe"" --type=renderer --autoplay-policy=no-user-gesture-required --disable-background-timer-throttling --field-trial-handle=1032,11197078799516264773,3373066149836992762,131072 --enable-features=ContextBridgeMutability,WebComponentsV0Enabled --disable-features=CookiesWithoutSameSiteMustBeSecure,SameSiteByDefaultCookies,SpareRendererForSitePerProcess --disable-gpu-compositing --lang=en-US --enable-wer --ms-teams-less-cors=522133263 --app-user-model-id=com.squirrel.Teams.Teams --app-path=""C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\resources\app.asar"" --device-scale-factor=1 --num-raster-threads=1 --renderer-client-id=338 --no-v8-untrusted-code-mitigations --mojo-platform-channel-handle=3892 /prefetch:1</Data><Data Name=""CurrentDirectory"">C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\</Data><Data Name=""User"">CYBERHUNTLABS\nmartha</Data><Data Name=""LogonGuid"">{16889A29-2C20-613F-201E-130000000000}</Data><Data Name=""Lo

<IPython.core.display.Javascript object>

### Python code can access the result by the _ variable
The underscore character is used by Python to hold and access the result of the KQL query from KQL Magic.
You can pass the result to a data frame df with df = _.to_dataframe()  


In [ ]:
df = _.to_dataframe()
df

### Present query output using the data frame df

In [32]:
df

NameError: name 'df' is not defined

### Show results in different forms

In [33]:
df.head()

NameError: name 'df' is not defined

In [ ]:
df.min()

In [ ]:
df.to_json()

In [ ]:
df.columns

### Excution of another cell overrides \_ variable.  
Use\_kql\_raw\_result\_ python variable holds last kql query result.  
If you run the previous cell you'll get an error since the \_ is now overrided


In [34]:
df = _kql_raw_result_

In [35]:
df

TenantId,SourceSystem,TimeGenerated,Source,EventLog,Computer,EventLevel,EventLevelName,ParameterXml,EventData,EventID,RenderedDescription,AzureDeploymentID,Role,EventCategory,UserName,Message,MG,ManagementGroupName,Type,_ResourceId
2d868894-04f8-4983-a155-36025f3ee696,OpsManager,2021-09-07 19:47:25.803000+00:00,Microsoft-Windows-Sysmon,Microsoft-Windows-Sysmon/Operational,win7.cyberhuntlabs.com,4,Information,"<Param>technique_id=T1036,technique_name=Masquerading</Param><Param>2021-09-07 19:47:25.795</Param><Param>{16889A29-C1CD-6137-1F0C-000000003500}</Param><Param>6048</Param><Param>C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe</Param><Param>1.4.00.22472</Param><Param>Microsoft Teams</Param><Param>Microsoft Teams</Param><Param>Microsoft Corporation</Param><Param>Teams.exe</Param><Param>""C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe"" --type=renderer --autoplay-policy=no-user-gesture-required --disable-background-timer-throttling --field-trial-handle=976,15683928964460437645,5454412970172637418,131072 --enable-features=ContextBridgeMutability,WebComponentsV0Enabled --disable-features=CookiesWithoutSameSiteMustBeSecure,SameSiteByDefaultCookies,SpareRendererForSitePerProcess --disable-gpu-compositing --lang=en-US --enable-wer --ms-teams-less-cors=522133263 --app-user-model-id=com.squirrel.Teams.Teams --app-path=""C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\resources\app.asar"" --device-scale-factor=1 --num-raster-threads=1 --renderer-client-id=696 --no-v8-untrusted-code-mitigations --mojo-platform-channel-handle=2748 /prefetch:1</Param><Param>C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\</Param><Param>CYBERHUNTLABS\nmartha</Param><Param>{16889A29-39C5-6137-B388-0D0000000000}</Param><Param>0xd88b3</Param><Param>2</Param><Param>Low</Param><Param>SHA256=AA09158C973672F5DC092403CBEA41A8E88DC598DCE1D4DC26A7B62C94C97CE5</Param><Param>{16889A29-39CE-6137-D600-000000003500}</Param><Param>4372</Param><Param>C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe</Param><Param>""C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe"" --system-initiated</Param>","<DataItem type=""System.XmlData"" time=""2021-09-07T19:47:25.8033838+00:00"" sourceHealthServiceId=""906B6E0A-5215-7CA5-3DF1-EFEE04E01033""><EventData xmlns=""http://schemas.microsoft.com/win/2004/08/events/event""><Data Name=""RuleName"">technique_id=T1036,technique_name=Masquerading</Data><Data Name=""UtcTime"">2021-09-07 19:47:25.795</Data><Data Name=""ProcessGuid"">{16889A29-C1CD-6137-1F0C-000000003500}</Data><Data Name=""ProcessId"">6048</Data><Data Name=""Image"">C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe</Data><Data Name=""FileVersion"">1.4.00.22472</Data><Data Name=""Description"">Microsoft Teams</Data><Data Name=""Product"">Microsoft Teams</Data><Data Name=""Company"">Microsoft Corporation</Data><Data Name=""OriginalFileName"">Teams.exe</Data><Data Name=""CommandLine"">""C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\Teams.exe"" --type=renderer --autoplay-policy=no-user-gesture-required --disable-background-timer-throttling --field-trial-handle=976,15683928964460437645,5454412970172637418,131072 --enable-features=ContextBridgeMutability,WebComponentsV0Enabled --disable-features=CookiesWithoutSameSiteMustBeSecure,SameSiteByDefaultCookies,SpareRendererForSitePerProcess --disable-gpu-compositing --lang=en-US --enable-wer --ms-teams-less-cors=522133263 --app-user-model-id=com.squirrel.Teams.Teams --app-path=""C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\resources\app.asar"" --device-scale-factor=1 --num-raster-threads=1 --renderer-client-id=696 --no-v8-untrusted-code-mitigations --mojo-platform-channel-handle=2748 /prefetch:1</Data><Data Name=""CurrentDirectory"">C:\Users\nmartha\AppData\Local\Microsoft\Teams\current\</Data><Data Name=""User"">CYBERHUNTLABS\nmartha</Data><Data Name=""LogonGuid"">{16889A29-39C5-6137-B388-0D0000000000}</Data><Data Name=""Logon

[{'TenantId': '2d868894-04f8-4983-a155-36025f3ee696', 'SourceSystem': 'OpsManager', 'TimeGenerated': datetime.datetime(2021, 9, 7, 19, 47, 25, 803000, tzinfo=tzutc()), 'Source': 'Microsoft-Windows-Sysmon', 'EventLog': 'Microsoft-Windows-Sysmon/Operational', 'Computer': 'win7.cyberhuntlabs.com', 'EventLevel': 4, 'EventLevelName': 'Information', 'ParameterXml': '<Param>technique_id=T1036,technique_name=Masquerading</Param><Param>2021-09-07 19:47:25.795</Param><Param>{16889A29-C1CD-6137-1F0C-000000003500}</Param><Param>6048</Param><Param>C:\\Users\\nmartha\\AppData\\Local\\Microsoft\\Teams\\current\\Teams.exe</Param><Param>1.4.00.22472</Param><Param>Microsoft Teams</Param><Param>Microsoft Teams</Param><Param>Microsoft Corporation</Param><Param>Teams.exe</Param><Param>"C:\\Users\\nmartha\\AppData\\Local\\Microsoft\\Teams\\current\\Teams.exe" --type=renderer --autoplay-policy=no-user-gesture-required --disable-background-timer-throttling --field-trial-handle=976,15683928964460437645,5454412970172637418,131072 --enable-features=ContextBridgeMutability,WebComponentsV0Enabled --disable-features=CookiesWithoutSameSiteMustBeSecure,SameSiteByDefaultCookies,SpareRendererForSitePerProcess --disable-gpu-compositing --lang=en-US --enable-wer --ms-teams-less-cors=522133263 --app-user-model-id=com.squirrel.Teams.Teams --app-path="C:\\Users\\nmartha\\AppData\\Local\\Microsoft\\Teams\\current\\resources\\app.asar" --device-scale-factor=1 --num-raster-threads=1 --renderer-client-id=696 --no-v8-untrusted-code-mitigations --mojo-platform-channel-handle=2748 /prefetch:1</Param><Param>C:\\Users\\nmartha\\AppData\\Local\\Microsoft\\Teams\\current\\</Param><Param>CYBERHUNTLABS\\nmartha</Param><Param>{16889A29-39C5-6137-B388-0D0000000000}</Param><Param>0xd88b3</Param><Param>2</Param><Param>Low</Param><Param>SHA256=AA09158C973672F5DC092403CBEA41A8E88DC598DCE1D4DC26A7B62C94C97CE5</Param><Param>{16889A29-39CE-6137-D600-000000003500}</Param><Param>4372</Param><Param>C:\\Users\\nmartha\\AppData\\Local\\Microsoft\\Teams\\current\\Teams.exe</Param><Param>"C:\\Users\\nmartha\\AppData\\Local\\Microsoft\\Teams\\current\\Teams.exe" --system-initiated</Param>', 'EventData': '<DataItem type="System.XmlData" time="2021-09-07T19:47:25.8033838+00:00" sourceHealthServiceId="906B6E0A-5215-7CA5-3DF1-EFEE04E01033"><EventData xmlns="http://schemas.microsoft.com/win/2004/08/events/event"><Data Name="RuleName">technique_id=T1036,technique_name=Masquerading</Data><Data Name="UtcTime">2021-09-07 19:47:25.795</Data><Data Name="ProcessGuid">{16889A29-C1CD-6137-1F0C-000000003500}</Data><Data Name="ProcessId">6048</Data><Data Name="Image">C:\\Users\\nmartha\\AppData\\Local\\Microsoft\\Teams\\current\\Teams.exe</Data><Data Name="FileVersion">1.4.00.22472</Data><Data Name="Description">Microsoft Teams</Data><Data Name="Product">Microsoft Teams</Data><Data Name="Company">Microsoft Corporation</Data><Data Name="OriginalFileName">Teams.exe</Data><Data Name="CommandLine">"C:\\Users\\nmartha\\AppData\\Local\\Microsoft\\Teams\\current\\Teams.exe" --type=renderer --autoplay-policy=no-user-gesture-required --disable-background-timer-throttling --field-trial-handle=976,15683928964460437645,5454412970172637418,131072 --enable-features=ContextBridgeMutability,WebComponentsV0Enabled --disable-features=CookiesWithoutSameSiteMustBeSecure,SameSiteByDefaultCookies,SpareRendererForSitePerProcess --disable-gpu-compositing --lang=en-US --enable-wer --ms-teams-less-cors=522133263 --app-user-model-id=com.squirrel.Teams.Teams --app-path="C:\\Users\\nmartha\\AppData\\Local\\Microsoft\\Teams\\current\\resources\\app.asar" --device-scale-factor=1 --num-raster-threads=1 --renderer-client-id=696 --no-v8-untrusted-code-mitigations --mojo-platform-channel-handle=2748 /prefetch:1</Data><Data Name="CurrentDirectory">C:\\Users\\nmartha\\AppData\\Local\\Microsoft\\Teams\\current\\</Data><Data Name="User">CYBERHUNTLABS\\nmartha</Data><Data Name="LogonGuid">{16889A29-39C5-6137-B388-0D0000000000}</Data><Data Name="Logo

### Alterantive way to assign kql query result to a python variable

In [106]:
%kql my_bar_chart << Event | where TimeGenerated >=ago(30d) | take 1000 | summarize count() by Source | sort by count_ | render barchart title='My bar chart'
my_bar_chart

<IPython.core.display.Javascript object>

[{'Source': 'Microsoft-Windows-Sysmon', 'count_': 731},
 {'Source': 'Microsoft-Windows-PowerShell', 'count_': 178},
 {'Source': 'PowerShell', 'count_': 82},
 {'Source': 'Service Control Manager', 'count_': 9}]

In [107]:
my_bar_chart.show_table()

Source,count_
Microsoft-Windows-Sysmon,731
Microsoft-Windows-PowerShell,178
PowerShell,82
Service Control Manager,9
